## Imports

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

## Data Reading

In [3]:
from catboost.datasets import titanic

# Data Reading
df_train, df_test = titanic()
df_train.set_index('PassengerId', inplace=True)
df_test.set_index('PassengerId', inplace=True)

# Split X_train, y_train
target = 'Survived'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

In [4]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing.base import *
from robusta.pipeline import *

nums = ['Age', 'Fare', 'SibSp', 'Parch']
cats = ['Pclass', 'Sex', 'Embarked']

data_prep = FeatureUnion([
        ("numeric", make_pipeline(
            ColumnSelector(nums),
            Imputer(strategy="median"),
            GaussRank(),
            #ColumnRenamer(prefix='gr_'),
        )),
        ("category", make_pipeline(
            ColumnSelector(cats),
            Imputer(strategy="most_frequent"),
            LabelEncoder(),
            #ColumnRenamer(prefix='le_'),
        )),
])

X_train = data_prep.fit_transform(df_train)
X_test = data_prep.transform(df_test)

X_train.head()

,Age,Fare,SibSp,Parch,Pclass,Sex,Embarked
PassengerId,,,,,,,
1,-0.488867,-0.968076,0.600843,-0.214091,2,1,2
2,0.584412,0.853865,0.600843,-0.214091,0,0,0
3,-0.275357,-0.454031,-0.290348,-0.214091,2,0,2
4,0.470091,0.703973,0.600843,-0.214091,0,0,2
5,0.470091,-0.379298,-0.290348,-0.214091,2,1,2


## Fold Preparation

In [5]:
from sklearn.model_selection import KFold
from robusta.resampler import *

encoder = FeatureUnion([
    ('category', make_pipeline(
        ColumnSelector(cats),
        TypeConverter('object'),
        TargetEncoderCV(cv=4).set_params(encoder__smoothing=200.0),
    )),
    ('numeric', make_pipeline(
        ColumnSelector(nums),
        Identity(),
    )),
])

resampler = SMOTE(random_state=50, k_neighbors=30)

fold_pipe = make_pipeline(resampler, encoder)

F_train = fold_pipe.fit_transform(X_train, y_train)

F_train.sample(5, random_state=555)

,Embarked,Pclass,Sex,Age,Fare,SibSp,Parch
928,0.453032,0.672597,0.785270,0.491782,1.040380,0.600843,-0.214091
873,0.457019,0.354773,0.273246,0.888832,-0.271711,-0.290348,-0.214091
293,0.460378,0.346921,0.800238,-0.381808,-0.274934,-0.290348,-0.214091
760,0.457019,0.354773,0.273246,-0.013926,0.016949,-0.290348,-0.214091
126,0.474911,0.326256,0.252564,-0.013926,-0.770943,-0.290348,-0.214091


## Model

In [6]:
%%time
from lightgbm import LGBMClassifier

model = LGBMClassifier()
model.fit(X_train, y_train)

CPU times: user 428 ms, sys: 7.35 ms, total: 435 ms
Wall time: 72.1 ms


/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [7]:
from sklearn.model_selection import cross_val_score

cv = 5
scoring = 'accuracy'

estimator = make_pipeline(model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(encoder, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [resampler + model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(resampler, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [encoder + model]'.format(np.mean(scores), np.std(scores)))

# 0.8216 ± 0.0370 (model)
# 0.8261 ± 0.0336 (resampler + model)
# 0.8249 ± 0.0320 (encoder + model)

0.8317 ± 0.0303 [model]
0.8306 ± 0.0212 [resampler + model]
0.8194 ± 0.0261 [encoder + model]


In [8]:
estimator = make_pipeline(encoder, model)

## Submit (Out-of-Fold & Test Predictions)

In [9]:
%%time
from robusta.crossval import cross_val_pred

y_oof, y_sub = cross_val_pred(estimator, cv, X_train, y_train, None, X_test, test_avg=False)

CPU times: user 17 s, sys: 2.81 s, total: 19.8 s
Wall time: 5.29 s


In [10]:
import os

path = os.path.join(os.getcwd(), 'pred')
if not os.path.exists(path):
    os.mkdir(path)

sub_path = os.path.join(path, '0 sub baseline.csv')
oof_path = os.path.join(path, '0 oof baseline.csv')

y_sub.to_csv(sub_path, header=True)
y_oof.to_csv(oof_path, header=True)

### Score:
### `[CV] 0.8249 ± 0.0314`
### `[LB] 0.7656`

In [11]:
from xgboost import XGBClassifier, XGBRanker
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from rgf import RGFClassifier, RGFRegressor

xgb = XGBClassifier()
lgb = LGBMClassifier()
#cat = CatBoostClassifier(n_estimators=100, verbose=0)
rgf = RGFClassifier()

In [14]:
#%%time
from robusta.crossval import stacking

y_oof, y_sub = stacking([xgb, lgb, rgf, estimator], 
                        cv, X_train, y_train, None, X_test, test_avg=False, 
                        method='predict_proba', n_jobs=-1)

In [15]:
y_oof

,XGB,LGBM,RGF,LGBM
PassengerId,,,,
1,0.081020,0.089169,0.098254,0.032297
2,0.987915,0.995316,0.983956,0.997331
3,0.444559,0.303481,0.369598,0.152854
4,0.985142,0.998688,0.992868,0.998502
5,0.191789,0.053231,0.180799,0.028786
6,0.141905,0.172941,0.134258,0.153592
7,0.210415,0.216723,0.319318,0.281467
8,0.337544,0.434587,0.439988,0.327672
9,0.457207,0.317327,0.381881,0.491998
